# SVM

## Load and import data

In [ ]:
from src.data.make_dataset import main as get_data
from src.data.preprocess import Preprocessor

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score,precision_score, recall_score, confusion_matrix, classification_report, roc_curve, roc_auc_score, f1_score
from sklearn import model_selection, svm
import scipy
import matplotlib.pyplot as plt
import pickle
import joblib

In [ ]:
raw_reviews = pd.read_csv("../../data/raw/reviews.csv")
raw_reviews.head()

In [ ]:
X_train, X_test, y_train, y_test = get_data(raw_reviews)

## Prepare data for SVM

In [ ]:
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train['cleaned_text'])
X_test_tfidf = vectorizer.transform(X_test['cleaned_text'])

X_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
X_test_tfidf = pd.DataFrame(X_test_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

X_train_clean = X_train.drop(['cleaned_text', 'text'], axis=1)
X_test_clean = X_test.drop(['cleaned_text', 'text'], axis=1)

X_train_concat = pd.concat([X_train_clean, X_train_tfidf], axis=1)
X_test_concat = pd.concat([X_test_clean, X_test_tfidf], axis=1)

X_train_concat = X_train_concat.loc[:, ~X_train_concat.columns.duplicated()].copy()
X_test_concat = X_test_concat.loc[:, ~X_test_concat.columns.duplicated()].copy()

In [ ]:
X_train_concat.shape #9213 features

# Vanilla SVM

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_concat,y_train)

### Evaluation

In [ ]:
# Make predictions on the testing set
y_pred = SVM.predict(X_test_concat)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1-score:", f1)
print("Confusion Matrix: \n", conf_matrix)
print("Classification Report: \n", class_report)

In [ ]:
# plot roc curve
fpr, tpr, _ = roc_curve(y_test,  y_pred)
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
auc = roc_auc_score(y_test, y_pred)
print(f"For this SVM model, the AUC score is: {auc}")

## Set max_features=5000 for TfidfVectorizer & refit into Vanilla SVM

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)

X_train_tfidf = vectorizer.fit_transform(X_train['cleaned_text'])
X_test_tfidf = vectorizer.transform(X_test['cleaned_text'])

X_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
X_test_tfidf = pd.DataFrame(X_test_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

X_train_clean = X_train.drop(['cleaned_text', 'text'], axis=1)
X_test_clean = X_test.drop(['cleaned_text', 'text'], axis=1)

X_train_concat = pd.concat([X_train_clean, X_train_tfidf], axis=1)
X_test_concat = pd.concat([X_test_clean, X_test_tfidf], axis=1)

X_train_concat = X_train_concat.loc[:, ~X_train_concat.columns.duplicated()].copy()
X_test_concat = X_test_concat.loc[:, ~X_test_concat.columns.duplicated()].copy()

# Vanilla SVM with max_features

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_concat,y_train)

### Evaluation

In [ ]:
# Make predictions on the testing set
y_pred = SVM.predict(X_test_concat)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
f1_score = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1-score:", f1_score)
print("Confusion Matrix: \n", conf_matrix)
print("Classification Report: \n", class_report) #better

In [ ]:
# plot roc curve
fpr, tpr, _ = roc_curve(y_test,  y_pred)
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
auc = roc_auc_score(y_test, y_pred)
print(f"For this SVM model, the AUC score is: {auc}")

# SVM with dim reduction (SVD: 100 components)

In [ ]:
n_components = 100
svd = TruncatedSVD(n_components=n_components)
X_train_svd = svd.fit_transform(X_train_tfidf)
X_test_svd = svd.transform(X_test_tfidf)

In [ ]:
X_train_svd_df = pd.DataFrame(X_train_svd, columns=[f'svd_{i}' for i in range(n_components)])
X_test_svd_df = pd.DataFrame(X_test_svd, columns=[f'svd_{i}' for i in range(n_components)])

X_train_concat_svd = pd.concat([X_train_clean, X_train_svd_df], axis=1)
X_test_concat_svd = pd.concat([X_test_clean, X_test_svd_df], axis=1)

In [ ]:
X_train_concat_svd.head()

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_concat_svd,y_train)

In [ ]:
# Make predictions on the testing set
y_pred = SVM.predict(X_test_concat_svd)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
f1_score = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1-score:", f1_score)
print("Confusion Matrix: \n", conf_matrix)
print("Classification Report: \n", class_report)

# Vanilla SVM with normalisation of FE feature

In [ ]:
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train['cleaned_text'])
X_test_tfidf = vectorizer.transform(X_test['cleaned_text'])

X_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
X_test_tfidf = pd.DataFrame(X_test_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

X_train_clean = X_train.drop(['cleaned_text', 'text'], axis=1)
X_test_clean = X_test.drop(['cleaned_text', 'text'], axis=1)

#added this
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_clean_scaled = scaler.fit_transform(X_train_clean)
X_train_clean_scaled = pd.DataFrame(X_train_clean_scaled, columns = X_train_clean.columns)
X_test_clean_scaled = scaler.fit_transform(X_test_clean)
X_test_clean_scaled = pd.DataFrame(X_test_clean_scaled, columns = X_train_clean.columns)

X_train_concat = pd.concat([X_train_clean_scaled, X_train_tfidf], axis=1)
X_test_concat = pd.concat([X_test_clean_scaled, X_test_tfidf], axis=1)

X_train_concat = X_train_concat.loc[:, ~X_train_concat.columns.duplicated()].copy()
X_test_concat = X_test_concat.loc[:, ~X_test_concat.columns.duplicated()].copy()

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_concat,y_train)

### Evaluation

In [ ]:
# Make predictions on the testing set
y_pred = SVM.predict(X_test_concat)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test,y_pred)
recall = recall_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
# print("f1-score:", f1)
print("Confusion Matrix: \n", conf_matrix)
print("Classification Report: \n", class_report)

In [ ]:
# plot roc curve
fpr, tpr, _ = roc_curve(y_test,  y_pred)
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()
auc = roc_auc_score(y_test, y_pred)
print(f"For this SVM model, the AUC score is: {auc}")

# Vanilla SVM with oversampling of minority class

In [ ]:
X_train_os, X_test_os, y_train_os, y_test_os = get_data("../data-dialogue/data/raw/reviews.csv", oversample=True)

In [ ]:
vectorizer = TfidfVectorizer(max_features=5000)

X_train_tfidf = vectorizer.fit_transform(X_train_os['cleaned_text'])
X_test_tfidf = vectorizer.transform(X_test_os['cleaned_text'])

X_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
X_test_tfidf = pd.DataFrame(X_test_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

X_train_clean = X_train_os.drop(['cleaned_text', 'text'], axis=1)
X_test_clean = X_test_os.drop(['cleaned_text', 'text'], axis=1)

X_train_concat = pd.concat([X_train_clean, X_train_tfidf], axis=1)
X_test_concat = pd.concat([X_test_clean, X_test_tfidf], axis=1)

X_train_concat = X_train_concat.loc[:, ~X_train_concat.columns.duplicated()].copy()
X_test_concat = X_test_concat.loc[:, ~X_test_concat.columns.duplicated()].copy()

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_concat,y_train_os)

In [ ]:
# Make predictions on the testing set
y_pred = SVM.predict(X_test_concat)

# Evaluate the model
accuracy = accuracy_score(y_test_os, y_pred)
precision = precision_score(y_test_os,y_pred)
recall = recall_score(y_test_os, y_pred)
conf_matrix = confusion_matrix(y_test_os, y_pred)
class_report = classification_report(y_test_os, y_pred)
# f1 = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
# print("f1-score:", f1)
print("Confusion Matrix: \n", conf_matrix)
print("Classification Report: \n", class_report)

# Evaluation:
best model - Vanilla SVM

In [ ]:
vectorizer = TfidfVectorizer()

X_train_tfidf = vectorizer.fit_transform(X_train['cleaned_text'])
X_test_tfidf = vectorizer.transform(X_test['cleaned_text'])

X_train_tfidf = pd.DataFrame(X_train_tfidf.toarray(), columns=vectorizer.get_feature_names_out())
X_test_tfidf = pd.DataFrame(X_test_tfidf.toarray(), columns=vectorizer.get_feature_names_out())

X_train_clean = X_train.drop(['cleaned_text', 'text'], axis=1)
X_test_clean = X_test.drop(['cleaned_text', 'text'], axis=1)

X_train_concat = pd.concat([X_train_clean, X_train_tfidf], axis=1)
X_test_concat = pd.concat([X_test_clean, X_test_tfidf], axis=1)

X_train_concat = X_train_concat.loc[:, ~X_train_concat.columns.duplicated()].copy()
X_test_concat = X_test_concat.loc[:, ~X_test_concat.columns.duplicated()].copy()

# Parameter Tuning

In [ ]:
param_grid = {'C': [0.1,1, 10], 
              'gamma': [1,0.1,0.01],
              'kernel': ['rbf', 'poly', 'sigmoid', 'linear']}

SVM = svm.SVC()
grid = GridSearchCV(SVM,param_grid,refit=True,verbose=1)
grid.fit(X_train_concat,y_train)

In [ ]:
print(grid.best_estimator_) #SVC(C=1, gamma=1, kernel='linear')

In [ ]:
grid_predictions = grid.predict(X_test_concat)

In [ ]:
print(confusion_matrix(y_test,grid_predictions))
print(classification_report(y_test,grid_predictions))#Output

# Evaluate the model
accuracy = accuracy_score(y_test, grid_predictions)
precision = precision_score(y_test,grid_predictions)
recall = recall_score(y_test, grid_predictions)
conf_matrix = confusion_matrix(y_test, grid_predictions)
class_report = classification_report(y_test, grid_predictions)
f1 = f1_score(y_test, grid_predictions)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("f1-score:", f1)
print("Confusion Matrix: \n", conf_matrix)
print("Classification Report: \n", class_report)

In [ ]:
# save best parameters
joblib.dump(grid.named_steps.model.best_estimator_, 'svm_best_model.pkl')

# Load Saved Model

In [ ]:
saved_model = joblib.load('svm_best_model.pkl')

# Add BaggingClassifier to speed up training

In [ ]:
model = BaggingClassifier(
            svm.SVC(C=1.0, kernel="linear", gamma=1, probability=True, random_state=4243),
            max_samples=0.3,
            n_estimators=10,
            n_jobs=-1,
            random_state=42,
        )

In [ ]:
model.fit(X_train_concat, y_train)

In [ ]:
model.predict(X_test_concat)